In [37]:
import pandas as pd
import numpy as np
data = pd.read_csv('data\\den_a_tochka.csv', encoding = 'cp1251', sep=';')

In [38]:
for col in data.columns:
    if data[col].isna().sum() == data.shape[0]:
        data.drop(columns=col, inplace=True)

drop_cols = [
    'Расценка(тариф)', 
    'КТУ',
    'Сумма',
    'Объемработ сучетомКТУ(н/час)',
    'Unnamed: 3',
    'Unnamed: 2',
    'Unnamed: 5'
    ]
data.drop(columns=drop_cols, inplace=True)

In [39]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5405 entries, 0 to 5404
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Наименование работ  5405 non-null   object
 1   Unnamed: 1          3858 non-null   object
 2   Unnamed: 4          2353 non-null   object
 3   Объемработ(н/час)   30 non-null     object
dtypes: object(4)
memory usage: 169.0+ KB


In [40]:
# Общий итог
print(data['Unnamed: 1'].iloc[-1]) 
# Убираем его
data = data.iloc[:-1]

3 050 929,000


In [41]:
#data[data['Unnamed: 2'].apply(lambda x: x==2350)]
#import numpy as np
#data[(data['Unnamed: 1'] != data['Unnamed: 5']) & (data['Unnamed: 1'].apply(lambda x: x is not np.nan))]
#data['Unnamed: 5'].value_counts()
data.fillna('0', inplace=True)
#data[(data['Unnamed: 1'] != data['Unnamed: 4']) & data['Unnamed: 1']=='0']
data['Цена'] = data['Unnamed: 1'].apply(lambda x: int(x.replace('\xa0','').split(',')[0])) + data['Объемработ(н/час)'].apply(lambda x: int(x.replace('\xa0','').split(',')[0]))
#data[data['Объемработ(н/час)']!='0']['Объемработ(н/час)']
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5404 entries, 0 to 5403
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Наименование работ  5404 non-null   object
 1   Unnamed: 1          5404 non-null   object
 2   Unnamed: 4          5404 non-null   object
 3   Объемработ(н/час)   5404 non-null   object
 4   Цена                5404 non-null   int64 
dtypes: int64(1), object(4)
memory usage: 211.2+ KB


In [42]:
drop_cols = ['Unnamed: 1', 'Unnamed: 4', 'Объемработ(н/час)']
data.drop(columns=drop_cols, inplace=True)

In [43]:
data

,Наименование работ,Цена
0,"Заказ-наряд №: 11 от 30.07.2021, ТС: Hyundai i...",0
1,"Заправка кондиц.(газ,масло)и экспресс диагност...",1700
2,Итого,1700
3,"Заказ-наряд №: 16 от 31.07.2021, ТС: ВАЗ 2109 ...",0
4,"Ведомая диагностика поиска неисправностей ,чте...",1000
...,...,...
5399,Электротехнические работы,2000
5400,Итого,2000
5401,"Заказ-наряд №: 8717 от 19.02.2023, ТС: Chevrol...",0
5402,Адаптация,500


In [44]:
data['Дата'] = pd.to_datetime(data['Наименование работ'].apply(lambda x: x[x.find('от')+3:x.find(',')] if 'Заказ-наряд №' in x else np.nan), dayfirst=True)
data['Дата'].fillna(method='ffill', inplace=True)

# HERE

In [64]:
# удаляем итоги и заказ-наряды
df_rabota = data[data['Наименование работ'].apply(lambda x: (x != 'Итого') and ('Заказ-наряд №' not in x))]
df_rabota.reset_index(inplace=True)
itogi = data['Цена'][data['Наименование работ'].apply(lambda x: x == 'Итого')]
#itogi.reset_index(inplace=True)
zakaz = data[['Наименование работ', 'Дата']][data['Наименование работ'].apply(lambda x: 'Заказ-наряд №' in x)]
#zakaz.reset_index(inplace=True)
df_zakaz = pd.concat([zakaz, itogi], axis=1, ignore_index=True)   # РАЗБЕРИСЬ С ИНДЕКСАМИ И КОНКАТЕНАЦИЕЙ!!!!!!!!!!!!!!!!

#                        HERE

In [62]:
itogi

2       1700
5       1000
8       1000
11      1000
14      8000
        ... 
5391    1500
5394     500
5397     300
5400    2000
5403     500
Name: Цена, Length: 1517, dtype: int64

In [65]:
df_zakaz

,0,1,2
0,"Заказ-наряд №: 11 от 30.07.2021, ТС: Hyundai i...",2021-07-30,NaN
3,"Заказ-наряд №: 16 от 31.07.2021, ТС: ВАЗ 2109 ...",2021-07-31,NaN
6,"Заказ-наряд №: 18 от 31.07.2021, ТС: ВАЗ 2114 ...",2021-07-31,NaN
9,"Заказ-наряд №: 19 от 31.07.2021, ТС: Toyota Ca...",2021-07-31,NaN
12,"Заказ-наряд №: 20 от 31.07.2021, ТС: Hyundai S...",2021-07-31,NaN
...,...,...,...
5391,NaN,NaT,1500.0
5394,NaN,NaT,500.0
5397,NaN,NaT,300.0
5400,NaN,NaT,2000.0


In [54]:
df_rabota

,Наименование работ,Цена,Дата
1,"Заправка кондиц.(газ,масло)и экспресс диагност...",1700,2021-07-30
4,"Ведомая диагностика поиска неисправностей ,чте...",1000,2021-07-31
7,"Ведомая диагностика поиска неисправностей ,чте...",1000,2021-07-31
10,Ремонт генератора,1000,2021-07-31
13,Электротехнические работы прошивка системы,8000,2021-07-31
...,...,...,...
5390,Ремонт(восстановление)электропроводки фар с за...,1500,2023-02-15
5393,Диагностика электрооборудования,500,2023-02-16
5396,Проверка ламп,300,2023-02-17
5399,Электротехнические работы,2000,2023-02-18
